In [39]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
from bokeh.io import push_notebook, show, output_notebook
import numpy as np
import time
from math import sin, radians, cos
output_notebook()

Loading BokehJS ...

In [54]:
def step(state, dt, box_bound):
    xy_new = state[ :2] + state[2: ] * dt + np.random.randn(2)
    while(True):
        if xy_new[0] < box_bound[0]:
            x_over = abs(box_bound[0] - xy_new[0])
            xy_new[0] = box_bound[0] + x_over
            state[2] *= -1
        elif xy_new[0] > box_bound[1]:
            x_over = abs(box_bound[1] - xy_new[0])
            xy_new[0] = box_bound[1] - x_over
            state[2] *= -1
        elif xy_new[1] < box_bound[2]:
            y_over = abs(box_bound[2] - xy_new[1])
            xy_new[1] = box_bound[2] + y_over
            state[3] *= -1
        elif xy_new[1] > box_bound[3]:
            y_over = abs(box_bound[3] - xy_new[1])
            xy_new[1] = box_bound[3] - y_over
            state[3] *= -1
        else: 
            state[:2] = xy_new
            break
    
    return state

In [55]:
def compute_hidden(velocity):
    if velocity[0] > 0 and velocity[1] > 0:
        return 0
    if velocity[0] > 0 and velocity[1] < 0:
        return 1
    if velocity[0] < 0 and velocity[1] < 0:
        return 2
    if velocity[0] < 0 and velocity[1] > 0:
        return 3

def transition(A, old_hidden, new_hidden):
    A[old_hidden, new_hidden] += 1
    return A
    

In [72]:
T = 100
dt = 1
init_state = np.random.random(4) * 10
# init_state = np.array([0,0,0.2,-0.1])

In [73]:
def generate_data(T, dt, init_state):
    A = np.zeros((4,4))
    STATE = np.zeros((T+1, 4))
    box_bound = np.array([0, 100, 0, 100])

    plot = figure(plot_width=300, plot_height=300)
    plot.x_range = Range1d(box_bound[0], box_bound[1])
    plot.y_range = Range1d(box_bound[2], box_bound[3])

    c = plot.circle(x=[init_state[0]], y=[init_state[1]])
    target = show(plot, notebook_handle=True)
    state = init_state
    STATE[0] = state
#     old_hidden = init_hidden(init_state[2:])
    for i in range(T):
        state = step(state, dt, box_bound)
#         print(state[2:])
        STATE[i+1] = state
        if i == 0:
            old_hidden = compute_hidden(state[2:])
        new_hidden = compute_hidden(state[2:])
        A = transition(A, old_hidden, new_hidden)
        old_hidden = new_hidden
        c.data_source.data['x'] = [state[0]]
        c.data_source.data['y'] = [state[1]]
        push_notebook(handle=target)
        time.sleep(0.2)
    Y = (STATE[1:] - STATE[:T])[:, :2]
    A = (A.T / A.sum(1)).T
    return Y, A

In [74]:
Y, A = generate_data(T, dt, init_state)

In [75]:
Y

array([[ 6.72852120e+00,  1.69217190e+00],
       [ 7.95689826e+00,  4.11669923e+00],
       [ 7.82045426e+00,  4.08369051e+00],
       [ 7.50796285e+00,  1.94779241e+00],
       [ 6.33459866e+00,  2.02524884e+00],
       [ 9.06717757e+00,  3.70301462e+00],
       [ 9.71311272e+00,  3.57256273e+00],
       [ 7.04295703e+00,  3.24153193e+00],
       [ 8.34858582e+00,  3.14053679e+00],
       [ 7.31511629e+00,  2.02364581e+00],
       [ 6.07808412e+00,  3.71106146e+00],
       [ 6.82420277e+00,  2.85277740e+00],
       [-1.24550386e+00,  3.83696756e+00],
       [-9.34830174e+00,  3.18385510e+00],
       [-6.09872672e+00,  3.20272101e+00],
       [-8.20778013e+00,  4.29033191e+00],
       [-5.87183855e+00,  3.65264212e+00],
       [-7.97945790e+00,  4.08338089e+00],
       [-6.89269078e+00,  2.71842850e+00],
       [-8.08518953e+00,  3.28686189e+00],
       [-7.76591853e+00,  3.79537729e+00],
       [-6.39154186e+00,  3.21868274e+00],
       [-6.38206754e+00,  3.27138041e+00],
       [-6.

In [38]:
C.sum(1)

array([1., 1., 1., 1.])